# One Hot Encoding and Training with LightGBM 

- In this first attempt I perform One Hot Encoding to the categorical features and ordinal encoding for the 'size' variable. Later I will train a LightGBM regressor with default parameters to predict The Price target variable.

In [4]:
import pandas as pd

In [ ]:
# If using local environment
df = pd.read_csv('cat_backpack.csv')

- One hot encoding for non hierarchycal features.

In [7]:

# Select the categorical columns to be encoded
categorical_cols = ['brand', 'material', 'style', 'color']

# Apply one-hot encoding using get_dummies
encoded_data = pd.get_dummies(df[categorical_cols], prefix=categorical_cols)

# Concatenate the encoded data with the original DataFrame
df = pd.concat([df, encoded_data], axis=1)

# Drop the original categorical columns
df.drop(categorical_cols, axis=1, inplace=True)

- Ordinal encoding for 'size' column.

In [8]:
# Create a dictionary to map size categories to numerical values
size_mapping = {
    'Small': 0,
    'Medium': 1,
    'Large': 2,
    'Unknown': 3  # Or you can assign it -1 or another distinct value
}

# Apply the mapping to the 'size' column
df['size_encoded'] = df['size'].map(size_mapping)

# Drop the original 'size' column (optional)
df.drop('size', axis=1, inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  300000 non-null  int64  
 1   compartments        300000 non-null  int64  
 2   laptop_compartment  300000 non-null  bool   
 3   waterproof          300000 non-null  bool   
 4   weight_cap          300000 non-null  float64
 5   Price               300000 non-null  float64
 6   brand_Adidas        300000 non-null  bool   
 7   brand_Jansport      300000 non-null  bool   
 8   brand_Nike          300000 non-null  bool   
 9   brand_Puma          300000 non-null  bool   
 10  brand_Under Armour  300000 non-null  bool   
 11  brand_Unknown       300000 non-null  bool   
 12  material_Canvas     300000 non-null  bool   
 13  material_Leather    300000 non-null  bool   
 14  material_Nylon      300000 non-null  bool   
 15  material_Polyester  300000 non-nul

# Train with LigthGBM.

In [10]:
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Drop the id column
df = df.drop(columns=['id'])

# Separate features and target variable
X = df.drop(columns=['Price'])
y = df['Price']

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the scaler and fit on training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create LightGBM datasets
train_data = lgb.Dataset(X_train_scaled, label=y_train)
test_data = lgb.Dataset(X_test_scaled, label=y_test, reference=train_data)

# Set up model parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'seed': 42
}

In [12]:
# Train the model
regressor = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[test_data],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)],  # Include log_evaluation callback
    # verbose_eval=100  # Remove this line
)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[41]	valid_0's rmse: 38.9136


In [14]:
# Predictions and evaluation
y_pred = regressor.predict(X_test_scaled, num_iteration=regressor.best_iteration)
# Calculate RMSE without the 'squared' argument, then take the square root
mse = mean_squared_error(y_test, y_pred)  # Remove squared=False
rmse = mse**0.5  # Calculate the square root to get RMSE
print("Test RMSE:", rmse)

Test RMSE: 38.91359184477554


# Fine Tune LightGBM with Optuna.

- In this section I will try to fine tune the lightGBM model using the optuna library.

In [15]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [16]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'random_state': 42,
        'n_estimators': 100,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
        'subsample': trial.suggest_float('subsample', 0.5, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0),
    }

    model = lgb.LGBMRegressor(**params)
    # Pass early stopping as a callback
    model.fit(
        X_train_scaled,
        y_train,
        eval_set=[(X_test_scaled, y_test)],
        callbacks=[lgb.early_stopping(stopping_rounds=10)]
    )
    predictions = model.predict(X_test_scaled)
    rmse = mean_squared_error(y_test, predictions)
    return rmse

In [17]:
import optuna
from optuna.samplers import TPESampler

sampler = TPESampler(seed=1) #  Uses the Tree-Structured Parzen Estimator algorithm [4]
study = optuna.create_study(direction='minimize', sampler = sampler)

[I 2025-02-15 14:20:36,669] A new study created in memory with name: no-name-b74c9da0-6f2a-4e58-9742-bccdfb7ed702


In [21]:
import time
start_time = time.time()
study.optimize(objective, n_trials=300)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training until validation scores don't improve for 10 rounds


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-02-15 15:45:39,147] Trial 100 finished with value: 1514.29393619688 and parameters: {'learning_rate': 0.056511548500071625, 'num_leaves': 48, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.7351574518948876, 'colsample_bytree': 0.7897765930147305, 'reg_alpha': 6.958497860857149, 'reg_lambda': 2.3726712919346844}. Best is trial 67 with value: 1513.6980983804954.


Early stopping, best iteration is:
[35]	valid_0's rmse: 38.9139


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

Did not meet early stopping. Best iteration is:
[92]	valid_0's rmse: 38.9071


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training until validation scores don't improve for 10 rounds


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-02-15 15:54:08,049] Trial 399 finished with value: 1513.9162116179878 and parameters: {'learning_rate': 0.07979087266612273, 'num_leaves': 26, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.8094189241845723, 'colsample_bytree': 0.7858023050376737, 'reg_alpha': 9.27816299096187, 'reg_lambda': 0.9732743240214802}. Best is trial 131 with value: 1513.596501576817.


Early stopping, best iteration is:
[64]	valid_0's rmse: 38.9091
Time taken: 509.70226979255676 seconds


In [22]:
print('Best parameters:', study.best_params)

Best parameters: {'learning_rate': 0.07814577909900611, 'num_leaves': 44, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.7921825554183903, 'colsample_bytree': 0.76774512355446, 'reg_alpha': 6.6527011724307155, 'reg_lambda': 1.484352269511822}


Best parameters: {'learning_rate': 0.09751747867142664, 'num_leaves': 72, 'max_depth': 4, 'min_child_samples': 57, 'subsample': 0.5096338621731877, 'colsample_bytree': 0.6570577434553186, 'reg_alpha': 3.7448144357284203, 'reg_lambda': 1.2766258463846785}

In [20]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Best parameters from Optuna
best_params = study.best_params
print("Best Parameters:", best_params)

# Create and train the model with the best parameters
model = lgb.LGBMRegressor(**best_params)
model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)],
          callbacks=[lgb.early_stopping(stopping_rounds=10)])

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
print("Test RMSE with Best Parameters:", rmse)

Best Parameters: {'learning_rate': 0.09751747867142664, 'num_leaves': 72, 'max_depth': 4, 'min_child_samples': 57, 'subsample': 0.5096338621731877, 'colsample_bytree': 0.6570577434553186, 'reg_alpha': 3.7448144357284203, 'reg_lambda': 1.2766258463846785}


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[65]	valid_0's l2: 1513.87
Test RMSE with Best Parameters: 38.90851707731672


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


- Best test rMSE metric achived after optimizing parameters with optuna was 38.9085, still far away from best performing model in competition :(